In [120]:
import struct
import numpy as np
import tensorflow as tf
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from keras.layers.merge import add, concatenate
from keras.models import Model

import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'

import numpy as np
import keras as k
import keras.backend as K
from keras.layers import Input, Lambda
from keras.optimizers import Adam

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

from functools import wraps
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2

from yolo3.utils import compose

In [121]:

def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = Conv2D(conv['filter'],
                   conv['kernel'],
                   strides=conv['stride'],           
                   padding='valid' 
                   if conv['stride'] > 1 else 'same',
                   name='conv_' + str(conv['layer_idx']),
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
    return add([skip_connection, x]) if skip else x
 

In [122]:
def make_yolov3_model(num_of_classes, num_of_anchors):
    input_image = Input(shape=(None, None, 3))

    res = (num_of_classes+5)*num_of_anchors
       
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
    
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                       {'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                       {'filter': 64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7},
                       {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 8}])
    

    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 10},
                   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 11},
                   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 12}])
    
    skip_2 = x
    
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 14},
                       {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 15},
                       {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16},
                       {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17}])

    skip_1 = x
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 18},
                       {'filter': 512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 19},
                       {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 20}])
    
    yolo_24 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 22},
                              {'filter': res, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 23}], skip=False)

    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 24}], skip=False)
    x = UpSampling2D(2)(x)
    
    print(x)
    print(skip_1)
    x = concatenate([x, skip_1])
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 26},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 27},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 30}], skip=False)

    yolo_33 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 31},
                              {'filter': res, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 32}], skip=False)
    # Layer 95 => 98
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 34}], skip=False)
    x = UpSampling2D(2)(x)
    
    print(x)
    print(skip_2)
    x = concatenate([x, skip_2])
    
    # Layer 99 => 106
    yolo_43 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 36},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 37},
                               {'filter': res, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 42}], skip=False)
    model = Model(input_image, [yolo_24, yolo_33, yolo_43]) 
    return model


def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)


In [123]:
annotation_path = '6_CLASS_test.txt'
log_dir = 'logs/000/'
classes_path = '6_CLASS_test_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
num_anchors = len(anchors)

input_shape = (416,416) # multiple of 32, hw
image_input = Input(shape=(None, None, 3))
# model = yolo_body(image_input,num_anchors//3,num_classes)
model_body = make_yolov3_model(num_classes,num_anchors//3)
h, w = input_shape

y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
    num_anchors//3, num_classes+5)) for l in range(3)]


model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])
model = Model([model_body.input, *y_true], model_loss)

Tensor("up_sampling2d_57/ResizeNearestNeighbor:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("add_114/add:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("up_sampling2d_58/ResizeNearestNeighbor:0", shape=(?, ?, ?, 256), dtype=float32)
Tensor("add_113/add:0", shape=(?, ?, ?, 256), dtype=float32)


In [124]:
logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
    
    
np.random.shuffle(lines)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

print(num_val)
print(num_train)

3354
30187


In [125]:
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred})
    
batch_size = 8
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=100,
        initial_epoch=0,
        callbacks=[logging, checkpoint])

Train on 30187 samples, val on 3354 samples, with batch size 8.
Epoch 1/100
3773/3773 [==============================] - 2511s 665ms/step - loss: 206.9817 - val_loss: 26.8248
Epoch 2/100
3773/3773 [==============================] - 2496s 662ms/step - loss: 21.5684 - val_loss: 19.3205
Epoch 3/100
3773/3773 [==============================] - 2496s 661ms/step - loss: 17.4350 - val_loss: 17.6362
Epoch 4/100
3773/3773 [==============================] - 2497s 662ms/step - loss: 15.6444 - val_loss: 15.3680
Epoch 5/100
3773/3773 [==============================] - 2503s 663ms/step - loss: 14.5093 - val_loss: 15.2081
Epoch 6/100
3773/3773 [==============================] - 2497s 662ms/step - loss: 13.7538 - val_loss: 13.6413
Epoch 7/100
3773/3773 [==============================] - 2500s 663ms/step - loss: 13.2737 - val_loss: 13.6079
Epoch 8/100
3773/3773 [==============================] - 2502s 663ms/step - loss: 12.9333 - val_loss: 13.1130
Epoch 9/100
3773/3773 [==============================] 

KeyboardInterrupt: 

In [ ]:
# model.save_weights(log_dir + 'trained_weights_stage_1.h5')
model.save("./compact_custom_model.h5")
model.save_weights("./compact_custom_weights.h5")

In [128]:
model.input

[<tf.Tensor 'input_106:0' shape=(?, ?, ?, 3) dtype=float32>,
 <tf.Tensor 'input_107:0' shape=(?, 13, 13, 3, 10) dtype=float32>,
 <tf.Tensor 'input_108:0' shape=(?, 26, 26, 3, 10) dtype=float32>,
 <tf.Tensor 'input_109:0' shape=(?, 52, 52, 3, 10) dtype=float32>]

In [130]:
model.output

<tf.Tensor 'yolo_loss_13/add_17:0' shape=() dtype=float32>